# TP 3: Scraping et APIs


Ce notebook reprend 2 exercices à faire chez vous. Chaque exercice vous demande de réaliser une série d'opérations. Utilisez les concepts vus au cours pour réaliser ce qui vous est demandé.

Nous passerons à travers les exercices lors de la séance de TP sur Teams. 


### 1. Utilisation d'APIs

- Cherchez sur Internet une API qui pourrait vous être utile dans le cadre de votre projet
- Faites un appel à cette API à travers un script Python et exportez le résultat dans le format de fichier adéquat (`txt`, `json`, `xlsx`, `csv`...)

Quelques sources d'inspiration:
- https://free-apis.github.io/#/categories
- https://www.odwb.be/pages/home/
- https://huggingface.co/docs/inference-providers/index 

Il en existe plein d'autres ! Soyez créatifs :-)

In [ ]:
import requests
from pprint import pprint
API_KEY = "dfc2e6777d40b9512ec6c937fc328736929dab6ffb0b186aadf86045"

headers = {
    "X-TextRazor-Key": API_KEY
}

data = {
    "extractors": "entities",
    "text": "L'UCLouvain est une université belge francophone située à Louvain-la-Neuve, en Wallonie"
}


response = requests.post("https://api.textrazor.com/", headers=headers, data=data)

result = []
for entity in response.json()["response"]["entities"]:
    print(entity["entityId"], entity["wikidataId"])
    
    parameters = {
        "action": "wbgetentities",
        "ids": entity["wikidataId"],
        "format": "json",
    }
    
    wikidata_response = requests.get(f"https://www.wikidata.org/w/api.php", params=parameters)
    name_nl = wikidata_response.json()["entities"][entity["wikidataId"]]["labels"]["nl"]["value"]
    result.append({
        "name": entity["entityId"],
        "wikidata_id": entity["wikidataId"],
        "name_nl": name_nl
    })

import pandas as pd
df = pd.DataFrame(result)
df.to_excel("entities.xlsx", index=False)
    


### 2. Scraping


Dans cet exercice, nous vous proposons de créer un répertoire de citations. Concrètement, vous allez extraire des citations depuis le site https://quotes.toscrape.com, un site spécialement conçu pour les exercices de scraping. Vous stockerez ensuite les données dans un fichier csv.

Voici les étapes à implémenter:

1. Ouvrir les 10 premières page du site avec `requests` (cela nécessitera sans doute une boucle)
2. A l'aide de `beautifulsoup`, Extraire chaque citation, avec les informations suivantes:
- L'auteur ou l'autrice
- Le texte de la citation
- La liste des tags associés
3. Stocker ces citations dans un DataFrame avec 3 colonnes
4. Exporter le DataFrame dans un fichier `csv`

In [ ]:

def get_textrazor_entities(text):

    headers = {
        "X-TextRazor-Key": API_KEY
    }

    data = {
        "extractors": "entities",
        "text": text
    }

    response = requests.post("https://api.textrazor.com/", headers=headers, data=data)
    wikidata_ids = []
    for entity in response.json()["response"].get("entities", []):
        wikidata_ids.append(entity.get("wikidataId"))
    return wikidata_ids
    


In [ ]:
import requests 
from bs4 import BeautifulSoup


quotes = []
for i in range(1, 3):
    
    print(f"Page {i}")
    response = requests.get(f"https://quotes.toscrape.com/page/{i}")
    soup = BeautifulSoup(response.text, "html.parser")

    for quote in soup.find_all("div", class_="quote"):
        text = quote.find("span", class_="text").text 
        author = quote.find("small", class_="author").text
        
        tags = [tag.text for tag in quote.find_all("a", class_="tag")]
        
        quotes.append({
            "text": text,
            "author": author,
            "tags": ", ".join(tags),
            "wikidata_ids": get_textrazor_entities(text)
        })

df = pd.DataFrame(quotes)
df.to_excel("quotes.xlsx", index=False)
    